# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize
from sklearn.model_selection import GridSearchCV

import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bignollo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Message", engine)
X = df['message']
Y = df.drop(["index","id","message","original","genre"], axis=1)

KeyError: "['index'] not found in axis"

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    words = word_tokenize(text)
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
y_pred = pd.DataFrame (y_pred, columns = y_test.columns)
for col in y_pred.columns:
    print(col + ":" + classification_report(y_test[col], y_pred[col]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        #'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, )
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# try other model -> RandomForestClassifier
rf_model = pipeline(clf = AdaBoostClassifier|())
rf_model.fit(X_train, y_train)
y_pred_rf_test = rf_model.predict(X_test)

print(classification_report(y_test.values, y_pred_rf_test, target_names=y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_3, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.